In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from datetime import date
import pandas as pd
import time

c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
c_driver = webdriver.Chrome(options=c_options)

c_driver.get('https://www.transfermarkt.com/')

In [21]:
years = list(range(2004, 2025))
top_leagues = {
    'Premier-League': 'GB1',
    'LaLiga': 'ES1',
    'Bundesliga': 'L1',
    'Serie-A': 'IT1',
    'Ligue-1': 'FR1'
}

In [22]:
teams_list = []
for k, v in top_leagues.items():
    for year in years:
        url = f'https://www.transfermarkt.com/{k.lower()}/startseite/wettbewerb/{v}/plus/?saison_id={year}'
        c_driver.get(url=url)
        time.sleep(8)
        teams_element = c_driver.find_elements(By.XPATH, '/html/body/div/main/div[1]/div[1]/div[2]/div[2]/div/table/tbody/tr')
        for team in teams_element:
            
            title_elements = team.find_elements(By.XPATH, './td[2]/a[position()>1]/img')
            titles = [title.get_attribute('alt') for title in title_elements]

            team_info = {
                'year': year,
                'league': k,
                'team': team.find_element(By.XPATH, './td[2]/a[1]').text,
                'squad_depth': team.find_element(By.XPATH, './td[3]/a').text,
                'avg_age': team.find_element(By.XPATH, './td[4]').text,
                'foreigners': team.find_element(By.XPATH, './td[5]').text,
                'avg_market_value': team.find_element(By.XPATH, './td[6]').text,
                'market_value': team.find_element(By.XPATH, './td[7]/a').text,
                'titles': titles
            }
            teams_list.append(team_info)

In [23]:
df = pd.DataFrame(teams_list)
df

,year,league,team,squad_depth,avg_age,foreigners,avg_market_value,market_value,titles
0,2004,Premier-League,Chelsea FC,31,24.9,24,€10.69m,€331.48m,[]
1,2004,Premier-League,Manchester United,37,24.7,25,€7.93m,€293.23m,[FA Cup Winner 03/04]
2,2004,Premier-League,Arsenal FC,37,23.9,29,€6.68m,€247.00m,[English Champion 03/04]
3,2004,Premier-League,Liverpool FC,38,25.3,26,€5.85m,€222.13m,[]
4,2004,Premier-League,Tottenham Hotspur,36,25.2,21,€3.53m,€127.20m,[]
...,...,...,...,...,...,...,...,...,...
2049,2024,Ligue-1,FC Toulouse,22,25.6,17,€3.73m,€82.15m,[]
2050,2024,Ligue-1,Le Havre AC,30,25.9,17,€2.35m,€70.45m,[]
2051,2024,Ligue-1,AJ Auxerre,28,25.7,17,€2.01m,€56.25m,[French 2nd tier champion 23/24]
2052,2024,Ligue-1,AS Saint-Étienne,27,26.9,12,€2.06m,€55.75m,[]


In [24]:
df.to_csv(f'../2004_2024_AllLeagues.csv')